# Training GameContextMLP (Expert)

## Imports

In [1]:
import sys
sys.path.append('../')

In [2]:
import pandas as pd
import numpy as np

In [11]:
from experts.mlp import GameContextMLP
from etl.dataloader import ExpertDataset
from torch.utils.data import DataLoader

## Loading Dataset

See `etl` module for more details.

In [4]:
# Make test-train-split
from sklearn.model_selection import train_test_split


df = pd.read_csv('features/target.csv')

# Split into train (80%) and test (20%) sets
df_train, df_test = train_test_split(df, test_size=0.2, random_state=42)

# Reset indices
df_train = df_train.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)


In [5]:
# create the dataset
MLP_train_dataset = ExpertDataset(df_train, expert_name="mlp")
MLP_val_dataset = ExpertDataset(df_test, expert_name="mlp")

In [6]:
# Get features and targets from dataset
X_train, y_train = MLP_train_dataset.features, MLP_train_dataset.targets
X_val, y_val = MLP_val_dataset.features, MLP_val_dataset.targets

In [7]:
X_train.shape, y_train.shape

((12465, 15), (12465,))

## Training expert
See `experts` module for more details.

In [9]:
mlp_model = GameContextMLP(input_size=X_train.shape[1])

In [10]:
# Training parameters
n_epochs = 50
batch_size = 32
learning_rate = 0.001

# Create dataloaders
train_loader = DataLoader(MLP_train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(MLP_val_dataset, batch_size=batch_size, shuffle=True)

# Train the model and get metrics
train_metrics = mlp_model.train_model(train_loader, val_loader,
                              epochs=n_epochs,
                              learning_rate=learning_rate)

# Get final predictions
train_preds = mlp_model.forward(X_train)
val_preds = mlp_model.forward(X_val)

# Calculate final accuracies
train_accuracy = np.mean(np.argmax(train_preds, axis=1) == y_train)
val_accuracy = np.mean(np.argmax(val_preds, axis=1) == y_val)

print(f"Final Training Accuracy: {train_accuracy:.4f}")
print(f"Final Validation Accuracy: {val_accuracy:.4f}")

TypeError: Module.train() got an unexpected keyword argument 'n_epochs'

In [ ]:

# Plot confusion matrices
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Training confusion matrix
plt.figure(figsize=(10,4))
plt.subplot(1,2,1)
train_cm = confusion_matrix(y_train, np.argmax(train_preds, axis=1))
sns.heatmap(train_cm, annot=True, fmt='d', cmap='Blues')
plt.title('Training Confusion Matrix')

# Validation confusion matrix 
plt.subplot(1,2,2)
val_cm = confusion_matrix(y_val, np.argmax(val_preds, axis=1))
sns.heatmap(val_cm, annot=True, fmt='d', cmap='Blues')
plt.title('Validation Confusion Matrix')
plt.tight_layout()
plt.show()